In [1]:
# Initialize the random number generator
using Random
rng = MersenneTwister(0);

In [2]:
# Size of the matrix
n = 512;

In [3]:
# Random initialization of matrix A
L = zeros(Float64,n,n)
U = zeros(Float64,n,n)
for i=1:n
    L[i,i] = 1
    L[i+1:n,i] = rand(rng, -2:2, n-i)
    U[i,i] = rand(rng, 1:2)
    U[i,i+1:n] = rand(rng, -2:2, n-i)
end
A = L * U
A0 = copy(A);
A0[1:6,1:6]

6×6 Array{Float64,2}:
  1.0  -1.0  -2.0   0.0   2.0  -1.0
 -2.0   4.0   3.0   1.0  -2.0   0.0
  0.0   2.0   0.0  -1.0   2.0  -2.0
 -1.0   1.0   1.0   3.0  -2.0   3.0
 -2.0  -2.0   7.0  -2.0  -7.0  10.0
  0.0   4.0  -1.0   2.0   4.0   1.0

In [4]:
# Initializing the right-hand side
xe = rand(rng, 0:9, n) # This will be our solution
b = A * xe
b'

1×512 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 168.0  -359.0  52.0  -265.0  -236.0  …  2344.0  -4226.0  492.0  1086.0

In [5]:
include("../src/getrf.jl")

getrs (generic function with 3 methods)

In [6]:
# Test our solvers
map([getrfOuter!, getrfAxpy!, getrfDot!]) do solver
    A = copy(A0)
    solver(A)
    # Solve
    x = getrs(A, b)
    @assert x == xe
    println(string(solver) * ": PASSED")
end    
;

getrfOuter!: PASSED
getrfAxpy!: PASSED
getrfDot!: PASSED


In [7]:
# Time the solvers
map([getrfOuter!, getrfAxpy!, getrfDot!]) do solver
    A = copy(A0); @time solver(A)
end
;

  0.284855 seconds
  0.043774 seconds
  0.121703 seconds
